## I’m Something of a Painter Myself
### Use GANs to create art - will you be the next Monet?



In [1]:
import tensorflow as tf
import os

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # Select the kaggle.json file from your system

# Move the file to the Kaggle folder
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c gan-getting-started

In [ ]:
!unzip gan-getting-started.zip -d ./gan_data

In [ ]:
monet_jpg_path = './gan_data/monet_jpg'
photo_jpg_path = './gan_data/photo_jpg'
monet_tfrec_path = './gan_data/monet_tfrec'
photo_tfrec_path = './gan_data/photo_tfrec'

monet_jpg = [os.path.join(monet_jpg_path, monet_images) for monet_images in os.listdir(monet_jpg_path)]
photo_jpg = [os.path.join(photo_jpg_path, photo_images) for photo_images in os.listdir(photo_jpg_path)]
monet_tfrec_files = [os.path.join(monet_tfrec_path, f) for f in os.listdir(monet_tfrec_path)]
photo_tfrec_files = [os.path.join(photo_tfrec_path, f) for f in os.listdir(photo_tfrec_path)]

print(f"Loaded {len(monet_jpg)} Monet jpg files.")
print(f"Loaded {len(photo_jpg)} Photo jpg files.")
print(f"Loaded {len(monet_tfrec_files)} Monet tfrec files.")
print(f"Loaded {len(photo_tfrec_files)} Photo tfrec files.")


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

monet_sample = Image.open(monet_jpg[0])
photo_sample = Image.open(photo_jpg[0])

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Monet Painting")
plt.imshow(monet_sample)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Photo")
plt.imshow(photo_sample)
plt.axis("off")

plt.show()

In [ ]:
#Function: help us to parse TFRecord data
def parse_tfrecord(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    parsed_example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.decode_jpeg(parsed_example['image'])
    return image

monet_dataset = tf.data.TFRecordDataset(monet_tfrec_files).map(parse_tfrecord)
photo_dataset = tf.data.TFRecordDataset(photo_tfrec_files).map(parse_tfrecord)

# Showing a Monet painting and a photo
for monet_image, photo_image in zip(monet_dataset.take(1), photo_dataset.take(1)):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Monet Painting")
    plt.imshow(tf.keras.utils.array_to_img(monet_image))
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Photo")
    plt.imshow(tf.keras.utils.array_to_img(photo_image))
    plt.axis("off")

    plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def downsample(layer_input, filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential([
        layers.Conv2D(filters, size, strides=2, padding='same',
                      kernel_initializer=initializer, use_bias=False)(layer_input),
        layers.BatchNormalization() if apply_batchnorm else layers.LayerNormalization(),
        layers.LeakyReLU()
    ])

    return result

def upsample(layer_input, filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential([
        layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                kernel_initializer=initializer, use_bias=False)(layer_input),
        layers.BatchNormalization(),
        layers.Dropout(0.5) if apply_dropout else layers.LayerNormalization(),
        layers.ReLU()
    ])
    return result


In [ ]:
import matplotlib.pyplot as plt

# Assume img is the loaded image tensor with size (height, width, channels)
def preprocess_image(image):
    image = tf.image.resize(image, [256, 256])
    image = tf.cast(image, tf.float32) / 127.5 - 1
    return image

monet_tensor = preprocess_image(monet_sample)

# Creating Downsample and Upsample layers
downsample_layer = downsample(filters=64, size=4)
upsample_layer = upsample(filters=64, size=4)

downsampled_image = downsample_layer(tf.expand_dims(monet_tensor, axis=0)) # Add batch dimension
downsampled_image = tf.squeeze(downsampled_image)  # Remove the batch dimension


upsampled_image = upsample_layer(tf.expand_dims(downsampled_image, axis=0))
upsampled_image = tf.squeeze(upsampled_image)

# Visualization
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow((monet_tensor + 1) / 2)


plt.imshow((downsampled_image[..., 0] + 1) / 2)
plt.title("Downsampled Image (Channel 0)")
plt.axis("off")
plt.show()

plt.subplot(1, 3, 3)
plt.title("Upsampled Image")
plt.imshow((upsampled_image[..., 0] + 1) / 2)
plt.axis("off")

plt.show()


In [ ]:
def Generator():
    OUTPUT_CHANNELS = 3
    inputs = tf.keras.Input(shape=[256,256,3])

    d1 = downsample(inputs,64,4,apply_batchnorm=False)
    d2 = downsample(d1,128,4)
    d3 = downsample(d2,256,4)
    d4 = downsample(d3,512,4)
    d5 = downsample(d4,512,4)
    d6 = downsample(d5,512,4)
    d7 = downsample(d6,512,4)
    d8 = downsample(d7,512,4)

    up1=upsample(d8,d7,512,4,apply_dropout=True)
    up2=upsample(up1,d6,512,4,apply_dropout=True)
    up3=upsample(up2,d5,512,4,apply_dropout=True)
    up4=upsample(up3,d4,512,4)
    up5=upsample(up4,d3,256,4)
    up6=upsample(up5,d2,128,4)
    up7=upsample(up6,d1,64,4)

    outputs = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh')(up7) # (batch_size, 256, 256, 3)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = layers.Input(shape=[256, 256, 3], name='input_image')

    x = inp

    down1 = downsample(64, 4, apply_batchnorm=False)(x)  # (bs, 128, 128, 64)
    down2 = downsample(128, 4)(down1)  # (bs, 64, 64, 128)
    down3 = downsample(256, 4)(down2)  # (bs, 32, 32, 256)

    zero_pad1 = layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
    conv = layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1)  # (bs, 31, 31, 512)

    norm1 = layers.LayerNormalization()(conv)  # LayerNormalization instead of InstanceNormalization
    leaky_relu = layers.LeakyReLU()(norm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)

    last = layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 30, 1)

    return tf.keras.Model(inputs=inp, outputs=last)

In [ ]:
class CycleGAN(tf.keras.Model):
    def __init__(self, monet_generator, photo_generator, monet_discriminator, photo_discriminator, lambda_cycle=10, lambda_identity=5):
        """
        Initialize CycleGAN model
        :param monet_generator: Monet style generator (photo -> Monet)
        :param photo_generator: Photo style generator (Monet -> photo)
        :param monet_discriminator: Monet discriminator
        :param photo_discriminator: Photo discriminator
        :param lambda_cycle: Weight of cycle consistency loss
        :param lambda_identity: Weight of identity mapping loss
        """
        super(CycleGAN, self).__init__()
        self.m_gen = monet_generator  # Photo -> Monet
        self.p_gen = photo_generator  # Monet -> Photo
        self.m_disc = monet_discriminator  # dicriminate Monet style
        self.p_disc = photo_discriminator  # dicriminate Photo style
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

        self.loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    def generator_loss(self, fake_output):
        return self.loss_obj(tf.ones_like(fake_output), fake_output)

    def discriminator_loss(self, real_output, fake_output):
        """
        Discriminator loss, distinguishing between real and generated images
        :param real_output: discriminator output for real images
        :param fake_output: discriminator output for generated images
        :return: loss value
        """
        real_loss = self.loss_obj(tf.ones_like(real_output), real_output)
        fake_loss = self.loss_obj(tf.zeros_like(fake_output), fake_output)
        return (real_loss + fake_loss) * 0.5

    def cycle_loss(self, real_image, cycled_image):
        """
        Cycle consistency loss
        :param real_image: original input image
        :param cycled_image: image that the generator cycles back to
        :return: loss value
        """
        return tf.reduce_mean(tf.abs(real_image - cycled_image)) * self.lambda_cycle

    def identity_loss(self, real_image, same_image):
        """
        Identity mapping loss
        :param real_image: original input image
        :param same_image: image directly generated by the generator
        :return: loss value
        """
        return tf.reduce_mean(tf.abs(real_image - same_image)) * self.lambda_cycle * self.lambda_identity

    def compile(self, m_gen_optimizer, p_gen_optimizer, m_disc_optimizer, p_disc_optimizer, gen_loss_fn, disc_loss_fn, cycle_loss_fn, identity_loss_fn):
        """
        CycleGAN
        """
        super(CycleGAN, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn

    def train_step(self, data):
        real_photo, real_monet = data

        with tf.GradientTape(persistent=True) as tape:
            # Forward passes
            fake_monet = self.m_gen(real_photo, training=True)
            cycled_photo = self.p_gen(fake_monet, training=True)

            fake_photo = self.p_gen(real_monet, training=True)
            cycled_monet = self.m_gen(fake_photo, training=True)

            same_photo = self.p_gen(real_photo, training=True)
            same_monet = self.m_gen(real_monet, training=True)

            disc_real_monet = self.m_disc(real_monet, training=True)
            disc_fake_monet = self.m_disc(fake_monet, training=True)

            disc_real_photo = self.p_disc(real_photo, training=True)
            disc_fake_photo = self.p_disc(fake_photo, training=True)

            # Loss calculations
            m_gen_loss = self.generator_loss(disc_fake_monet)
            p_gen_loss = self.generator_loss(disc_fake_photo)

            cycle_loss = self.cycle_loss(real_photo, cycled_photo) + self.cycle_loss(real_monet, cycled_monet)
            identity_loss = self.identity_loss(real_photo, same_photo) + self.identity_loss(real_monet, same_monet)

            total_m_gen_loss = m_gen_loss + cycle_loss + identity_loss
            total_p_gen_loss = p_gen_loss + cycle_loss + identity_loss

            m_disc_loss = self.discriminator_loss(disc_real_monet, disc_fake_monet)
            p_disc_loss = self.discriminator_loss(disc_real_photo, disc_fake_photo)

        # Apply gradients
        m_gen_gradients = tape.gradient(total_m_gen_loss, self.m_gen.trainable_variables)
        p_gen_gradients = tape.gradient(total_p_gen_loss, self.p_gen.trainable_variables)

        m_disc_gradients = tape.gradient(m_disc_loss, self.m_disc.trainable_variables)
        p_disc_gradients = tape.gradient(p_disc_loss, self.p_disc.trainable_variables)

        self.m_gen_optimizer.apply_gradients(zip(m_gen_gradients, self.m_gen.trainable_variables))
        self.p_gen_optimizer.apply_gradients(zip(p_gen_gradients, self.p_gen.trainable_variables))

        self.m_disc_optimizer.apply_gradients(zip(m_disc_gradients, self.m_disc.trainable_variables))
        self.p_disc_optimizer.apply_gradients(zip(p_disc_gradients, self.p_disc.trainable_variables))

        # Return loss dictionary
        return {
            "Monet_Generator_Loss": total_m_gen_loss,
            "Photo_Generator_Loss": total_p_gen_loss,
            "Monet_Discriminator_Loss": m_disc_loss,
            "Photo_Discriminator_Loss": p_disc_loss,
        }


In [ ]:
monet_generator = Generator()  # Monet-style generator
photo_generator = Generator()  # Photo-style generator
monet_discriminator = Discriminator()  # Monet discriminator
photo_discriminator = Discriminator()  # Photo discriminator

monet_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
photo_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
monet_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
photo_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

cycle_gan_model = CycleGAN(monet_generator, photo_generator, monet_discriminator, photo_discriminator)

cycle_gan_model.compile(
    m_gen_optimizer=monet_generator_optimizer,
    p_gen_optimizer=photo_generator_optimizer,
    m_disc_optimizer=monet_discriminator_optimizer,
    p_disc_optimizer=photo_discriminator_optimizer,
    gen_loss_fn=CycleGAN.generator_loss,
    disc_loss_fn=CycleGAN.discriminator_loss,
    cycle_loss_fn=CycleGAN.cycle_loss,
    identity_loss_fn=CycleGAN.identity_loss
)

In [ ]:

def load_image(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, [256, 256])
    image = (image / 127.5) - 1  # Normalize to [-1, 1]
    return image

def load_monet(image_file):
    return load_image(image_file)

def load_photo(image_file):
    return load_image(image_file)

# Convert .jpg file paths to a TensorFlow dataset
monet_ds = tf.data.Dataset.from_tensor_slices(monet_jpg)
monet_ds = monet_ds.map(load_monet, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(1)

photo_ds = tf.data.Dataset.from_tensor_slices(photo_jpg)
photo_ds = photo_ds.map(load_photo, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(1)


In [ ]:
#Train the CycleGAN model
# Combine datasets
dataset = tf.data.Dataset.zip((photo_ds, monet_ds))

# Fit the model
cycle_gan_model.fit(
    dataset,
    epochs=25
)

In [ ]:
import numpy as np
_, ax = plt.subplots(5, 2, figsize=(20, 20))

for i, img in enumerate(photo_ds.take(5)):
    prediction = monet_generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)  # De-normalize the generated image
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)  # De-normalize the input photo

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input Photo")
    ax[i, 1].set_title("Monet-esque")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")

plt.show()

In [ ]:
# Get the history object
history = cycle_gan_model.history

# Extract loss values
monet_gen_loss = history.history['Monet_Generator_Loss']
photo_gen_loss = history.history['Photo_Generator_Loss']
monet_disc_loss = history.history['Monet_Discriminator_Loss']
photo_disc_loss = history.history['Photo_Discriminator_Loss']

# Plot generator and discriminator losses over epochs
plt.figure(figsize=(10, 6))

plt.plot(monet_gen_loss, label='Monet Generator Loss')
plt.plot(photo_gen_loss, label='Photo Generator Loss')
plt.plot(monet_disc_loss, label='Monet Discriminator Loss')
plt.plot(photo_disc_loss, label='Photo Discriminator Loss')

plt.title('CycleGAN Training Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
import PIL
! mkdir ../images
import shutil

i = 1
for img in photo_ds:
    print('Index:', i)
    prediction = monet_generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    im.save("../images/" + str(i) + ".jpg")
    i += 1

shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")